1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


Думаю, выбор метрики зависит от задачи. Т.к. hit_rate покажет только подошла ли хоть одна рекомендация или нет, а precision покажет долю правильных рекомендаций.
Насколько я понял, hit_rate используем там, где нам не важно количество купленых товаров/услуг, возможно, потому что они равнозначны и нам достаточно, если клиент выберет 1 из вариантов.
Примеры hit_rate:
1. Рекомендуем клиенту приобрести пиццу на выбор из 3ех вариантов. Клиент - клиент, пицца - товар. 
2. Предлагаем клиенту посетить один из ресторанов сети. Клиент - клиент, ресторан - товар.
3. Предлагаем прослушать музыкальный плейлист из ежедневных рекомендаций. Клиент - клиент, плейлист - товар.

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Используем логарифм, т.к. функция гладкая, т.е. имеет производные на всем своем отрезке. Можно использовать 

$$y = x^3 $$

Так же логарифм используется в logloss

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Так же возможно посчитать мерику f1@k основываясь на precision@k и recall@k.

$$ F1@k = 2 * ((precision@k * recall@k)/(precision@k + recall@k)) $$

Можем использовать ее, когда не можем знать, что важнее recall@k или precision@k.


4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

In [1]:
import numpy as np

In [8]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

##### PR@8

In [126]:
pr8 = 4/8
pr8

0.5

In [4]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [9]:
pr_8 = precision_at_k(recommended, boughted, 8)
pr_8

0.5

##### rec@8

In [125]:
rec8 = 4/6
rec8

0.6666666666666666

In [5]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [10]:
rec_8 = recall_at_k(recommended, boughted, 8)
rec_8

0.6666666666666666

##### AP@8

In [122]:
ap8 = 1/10 * (0 + 0 + 1/3 + 2/4 + 3/5 + 4/6 + 0 + 0)
ap8

0.20999999999999996

In [123]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
    
    
    result = sum_ / k
    
    return result

In [124]:
ap_8 = ap_k(recommended, boughted, 8)
ap_8

0.2

##### NDCG@8

In [69]:
dcg1 = 1/8 * (0/1 + 1/np.log(2) + 1/np.log(3) + 0/np.log(4) + 1/np.log(5) + 1/np.log(6) + 0/np.log(7) + 0/np.log(8))
dcg1

0.44154747857833254

In [70]:
ideal_dcg1 = 1/8 * (1/1 + 1/np.log(2) + 1/np.log(3) + 1/np.log(4) + 1/np.log(5) + 1/np.log(6) + 1/np.log(7) + 1/np.log(8))
ideal_dcg1

0.7810655048004851

In [77]:
ndcg1 = dcg1/ideal_dcg1
ndcg1

0.5653142737255068

In [108]:
def dcg(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)

    sum_ = 0
    for i in range(0, k): 
        if i == 0 and flags[i] == True: 
            sum_ += 1
        elif flags[i] == True:
            sum_ += 1/np.log(i+1)
    
    return sum_ / k
  

In [109]:
dcg(recommended, boughted, 8)

0.44154747857833254

In [110]:
def ideal_dcg(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    sum_ = 0
    for i in range(1, k+1): 
        if i == 1:
            sum_ += 1
        else:
            sum_ += 1/np.log(i)
    
    return sum_ / k

In [111]:
ideal_dcg(recommended, boughted, 8)

0.7810655048004851

In [112]:
def ndcg(recommended_list, bought_list, k):
    return dcg(recommended_list, bought_list, k)/ideal_dcg(recommended_list, bought_list, k)

In [113]:
ndcg(recommended, boughted, 8)

0.5653142737255068

##### RR@8

In [142]:
boughted

[1, 3, 5, 7, 9, 11]

In [143]:
recommended

[2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

In [176]:
rr8 = 1/2
rr8

0.5

In [174]:
def reciprocal_rank(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    results = []
    for i in range(0, k):
        if flags[i] == True:
            x = 1/(i+1)
            results.append(x)
    
    return results[0]
    

In [175]:
reciprocal_rank(recommended, boughted, 8)

0.5

##### ERR@8

In [199]:
def err(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    # считаем ранги для всех
    ranks = []
    for i in range(0, k):
        if flags[i] == True:
            x = i+1
            ranks.append(x)
    
                        
    p = 1 # изначальная вероятность
    ERR = 0
    
    for i in range(0, len(ranks)) :
        R = (2^ranks[i] - 1)/(2^(np.max(ranks)))
        ERR = ERR + p * (R/ranks[i])
        p = p * (1 - R)
                   
    return ERR
        
        

In [200]:
err(recommended, boughted, 8)

0.4135416666666667

С err совсем не уверен, что посчитал то, что нужно. 
Руководствовался статьей https://web.archive.org/web/20200426192936/http://www.olivier.chapelle.cc/pub/err.pdf , но не уверен, что правильно во всем разобрался.